In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/working/check_point'):
    for filename in filenames:
        os.remove(os.path.join(dirname, filename))

In [8]:
pip install thop

Note: you may need to restart the kernel to use updated packages.


In [9]:
import torch
import numpy as np
from torchvision.datasets import ImageFolder
from torchvision import models, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.nn as nn
import time
from tqdm import tqdm
import sys
import torch.nn.functional as F
from torch.nn.functional import cross_entropy
import os
from thop import profile
import pandas as pd
import glob
import math
import torch.optim.lr_scheduler as lr_scheduler

# 配置运行设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 设置参数
batch_size = 32
learning_rate = 0.00003
num_epoch = 50
pretrained = False
only_train_fc = False
model_name = 'resnet18'

In [10]:
# 参数, 模型
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 32
learning_rate = 0.00005
num_epoch = 50
pretrained = False
only_train_fc = False
model_name = 'resnet18'

In [11]:
# 数据处理
normalize = transforms.Normalize(mean=[0, 0, 0], std=[1, 1, 1])
transform = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
    normalize
])

# 读取图像数据
train_dataset = ImageFolder('/kaggle/input/breakhis-total-dataset/train/', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = ImageFolder('/kaggle/input/breakhis-total-dataset/test/', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

print('{0} for train. {1} for val'.format(len(train_dataset), len(test_dataset)))

8486 for train. 2122 for val


In [12]:
save_path = '/kaggle/working/check_point'
if not os.path.exists(save_path):
    os.mkdir(save_path)
result_path = os.path.join('/kaggle/working/check_point', model_name)
if not os.path.exists(result_path):
    print(1)
    os.mkdir(result_path)
if len(glob.glob(result_path + '/**.pth')) == 0:
    model = models.resnet18()
    fc_in_features = model.fc.in_features
    model.fc = torch.nn.Linear(fc_in_features, 2, bias=True)
else:
    model_path = glob.glob(result_path + '/**.pth')[-1]
    model = torch.load(model_path)
if only_train_fc:
    for param in model.parameters():
        param.requires_grad_(False)
train_test_data_path = os.path.join(result_path, 'train_test_data.csv')
if os.path.exists(train_test_data_path):
    train_test_data = pd.read_csv(train_test_data_path)
    last_epoch = train_test_data.shape[0]
    test_acc_best = train_test_data['test_acc_best'].values[-1]
else:
    train_test_data = pd.DataFrame(data=[], columns=['train_acc', 'train_loss',
                                                     'train_lr',
                                                     'test_acc', 'test_loss', 
                                                     'epoch', 'test_acc_best'])
    last_epoch = 0
    test_acc_best = 0
assert num_epoch > last_epoch, '已达训练次数'

In [13]:
model = model.to(device)
# 输出模型参数与模型计算量

flops, params = profile(model, inputs=(torch.zeros((batch_size, 3, 224, 224)).to(device),), verbose=False)
print(f'number of parameter: {params}', ', %.1f GFLOPS' % (flops / 1E9 * 2))


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss(reduction='sum')

def get_parameter_number(model):
    total_num = sum(p.numel() for p in model.parameters())
    trainable_num = sum(p.numel() for p in model.parameters()if p.requires_grad)
    return {'Total': total_num, 'Trainable': trainable_num}
print(get_parameter_number(model))

number of parameter: 11177538.0 , 116.7 GFLOPS
{'Total': 11177538, 'Trainable': 11177538}


In [ ]:
# 动态学习率
# def lf(epoch):
#     # 前6个epoch学习率保持不变，个epoch后学习率按比例衰减
#     if epoch <= 5:
#         return 1
#     else:
#         lr = math.exp(0.1 * (5 - epoch))
#         return lr
# scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lf)
# scheduler.last_epoch = last_epoch -1
def train(epoch, dataloder):
    model.train()

    t0 = time.time()
    for (X_batch, y_batch) in tqdm(dataloder, leave=False, desc=f'epoch:{epoch}'):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()

        out_put = model(X_batch)
        loss = loss_fn(out_put, y_batch)
        loss.backward()
        optimizer.step()
    now_lr = optimizer.param_groups[0]["lr"]
#     scheduler.step()
    t1 = time.time()
    print(f'epoch[{epoch}] time[{round(t1 - t0, 1)}]s lr:{now_lr}')
    return now_lr

def evaluate(data_loader, type):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        eval_loss = 0
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(F.softmax(outputs, dim=-1), 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            eval_loss += loss_fn(outputs, labels).item()
    acc = 100 * correct / total
    print(f'Accuracy on {type} set: {round(acc, 2)}%  eval_loss:{eval_loss / total}')
    return acc, eval_loss / total


'''
每次运行之后要保存：
1.模型本身
2.模型迭代次数
3.模型学习率
4.历代模型表现情况loss,acc等

'''


if __name__ == '__main__':
    weight_path_best = os.path.join(result_path, 'best')
    if not os.path.exists(weight_path_best):
        os.mkdir(weight_path_best)
    for epoch in range(last_epoch, num_epoch):
        weight_name = '{}-oral'.format(model_name)
        lr = train(epoch, train_loader)
        train_acc, train_loss = evaluate(train_loader, type='train')
        test_acc, test_loss = evaluate(test_loader, type='test')
        
        # 保存训练好的模型之前，删掉已有的模型
        for f in glob.glob(result_path + '/**.pth'):
            os.remove(f)
        weight_name = weight_name + '_epoch{}.pth'.format(epoch + 1)
        weight_path = os.path.join(result_path, weight_name)
        torch.save(model, weight_path)
        
        if test_acc > test_acc_best:
            test_acc_best = test_acc
            for f in glob.glob(weight_path_best + '/**.pth'):
                os.remove(f)
            torch.save(model, os.path.join(weight_path_best, weight_name))
        train_test_data.loc[len(train_test_data.index)] = [train_acc, train_loss,
                                                           lr,
                                                           test_acc, test_loss,
                                                           epoch, test_acc_best]

        train_test_data.to_csv(train_test_data_path, index=False)

    train_acc_array = train_test_data['train_acc'].values
    test_acc_array = train_test_data['test_acc'].values
    train_loss_array = train_test_data['train_loss'].values
    test_loss_array = train_test_data['test_loss'].values

    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(train_acc_array, label='Training Accuracy')
    plt.plot(test_acc_array, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.title('Capsnet Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(train_loss_array, label='Training Loss')
    plt.plot(test_loss_array, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Margin Entropy')
    plt.title('Capsnet Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

epoch[8] time[189.1]s lr:5e-05
Accuracy on train set: 97.84%  eval_loss:0.05365206400358766
Accuracy on test set: 85.44%  eval_loss:0.4810402112452069


epoch[9] time[132.8]s lr:5e-05
Accuracy on train set: 89.92%  eval_loss:0.35011604073585384
Accuracy on test set: 79.78%  eval_loss:1.1039777757085572


epoch[10] time[132.3]s lr:5e-05
Accuracy on train set: 97.47%  eval_loss:0.06683762568472103
Accuracy on test set: 84.68%  eval_loss:0.5619468451894531


epoch[11] time[131.2]s lr:5e-05
Accuracy on train set: 99.12%  eval_loss:0.02418808120010871
Accuracy on test set: 88.12%  eval_loss:0.44420923364263565


epoch[12] time[132.8]s lr:5e-05
Accuracy on train set: 99.94%  eval_loss:0.0041700650296511215
Accuracy on test set: 88.69%  eval_loss:0.41992774105066405


epoch[13] time[135.1]s lr:5e-05
Accuracy on train set: 98.88%  eval_loss:0.03154148099599428
Accuracy on test set: 87.61%  eval_loss:0.5577975910975055


epoch[14] time[133.3]s lr:5e-05
Accuracy on train set: 98.62%  eval_loss:0.04293966058295636
Accuracy on test set: 85.91%  eval_loss:0.5574682292569706


epoch[15] time[132.0]s lr:5e-05
Accuracy on train set: 99.2%  eval_loss:0.021752444509879516
Accuracy on test set: 87.04%  eval_loss:0.5300272372845318


epoch[16] time[131.7]s lr:5e-05
Accuracy on train set: 97.55%  eval_loss:0.128585626713619
Accuracy on test set: 86.43%  eval_loss:0.7289122661591475


epoch[17] time[133.0]s lr:5e-05
Accuracy on train set: 99.42%  eval_loss:0.016202241062427444


In [ ]:
!pip install torchviz

In [17]:
from torchviz import make_dot
x = torch.randn(1, 3, 224, 224).requires_grad_(True).to(device)  # 定义一个网络的输入值
y = model(x)    # 获取网络的预测值

MyConvNetVis = make_dot(y, params=dict(list(model.named_parameters()) + [('x', x)]))
MyConvNetVis.format = "png"
# 指定文件生成的文件夹
MyConvNetVis.directory = "/kaggle/working/check_point"
# 生成文件
MyConvNetVis.view()

'/kaggle/working/check_point/Digraph.gv.png'

Error: no "view" mailcap rules found for type "image/png"
/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening '/kaggle/working/check_point/Digraph.gv.png'


In [22]:
!ls /usr/bin/xdg-open

/usr/bin/xdg-open


In [19]:
from PIL import Image
img=Image.open('/kaggle/working/check_point/Digraph.gv.png')
img.show()

Error: no "view" mailcap rules found for type "image/png"
/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening '/tmp/tmpp6nrfpnj.PNG'
